# 1. 패키지 설치

In [ ]:
# poetry add pypdf=">=4.2.0,<5.0.0"
# poetry add langchain-upstage

# 2. 환경변수 불러오기

- `.env` 파일에 `UPSTAGE_API_KEY` 등록

In [1]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
print(UPSTAGE_API_KEY[:4])

up_X


# 3. LLM 답변 생성

- Upstage Console에서 발급받은 API Key를 `UPSTAGE_API_KEY`라고 저장하면 별도의 설정 없이 `ChatUpstage`를 사용할 수 있음

In [2]:
from langchain_upstage import ChatUpstage

#llm = ChatUpstage(temperature=0.5)
llm = ChatUpstage(
        model="solar-pro",
        base_url="https://api.upstage.ai/v1",
        temperature=0.5
    )

In [3]:
ai_message=llm.invoke("LangChain은 무엇인가요?")
print(type(ai_message))
print(ai_message.content)

<class 'langchain_core.messages.ai.AIMessage'>
**LangChain**은 대규모 언어 모델(LLM, Large Language Model)을 활용해 애플리케이션 개발을 용이하게 하는 **오픈소스 프레임워크**입니다. LLM의 강력한 기능을 실제 서비스에 통합할 때 발생하는 복잡성(예: 컨텍스트 관리, 도구 연동, 메모리 처리 등)을 해결하기 위해 설계되었습니다.

### 📌 **LangChain의 핵심 개념**
1. **모듈식 설계**  
   - LLM, 프롬프트, 메모리, 검색, 체인(Chain), 에이전트(Agent) 등을 조합해 유연하게 시스템을 구축할 수 있습니다.
   - 예: `Prompt Template + LLM + Retrieval`을 연결해 QA 시스템 생성.

2. **주요 구성 요소**  
   - **모델 I/O**: OpenAI, Anthropic, HuggingFace 등 다양한 LLM과 호환.
   - **체인(Chain)**: 여러 컴포넌트를 연결해 복잡한 작업 처리 (예: 요약 → 번역 → 저장).
   - **에이전트(Agent)**: LLM이 동적으로 도구를 선택해 작업 수행 (예: 계산기, 검색 엔진 호출).
   - **메모리(Memory)**: 대화 기록을 유지해 컨텍스트 지속 (챗봇에 유용).
   - **데이터 연결**: 벡터 DB(Pinecone, FAISS), API, SQL 등과 연동.

3. **사용 사례**  
   - **챗봇**: 대화 기록과 외부 지식 검색 결합.
   - **자동화된 문서 처리**: PDF 분석 → 요약 → 데이터베이스 저장.
   - **코드 생성**: GitHub API와 연동해 코드 추천.

### 🛠 **LangChain의 장점**
- **생산성 향상**: 복잡한 파이프라인을 코드로 간단히 구현.
- **확장성**: 커스텀 모듈 추가 가능 (예: 자체 LLM 통합).
- **커뮤니티 지원**: 활발한 생태계로 튜토리얼과 예제 풍부.

##

In [ ]:
# using chat stream
for chunk in llm.stream("LangChain은 무엇인가요?"):
    print(chunk.content, end="")

In [ ]:
from langchain_upstage import ChatUpstage
from langchain_core.prompts import ChatPromptTemplate

translation_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a professional translator specializing in Korean-English translation."),
        ("human", "Translate this from {source_lang} to {target_lang}: {text}")
    ])

llm = ChatUpstage(
        model="solar-pro",
        base_url="https://api.upstage.ai/v1",
        temperature=0.5
    )

# 체인 실행
chain = translation_prompt | llm

response = chain.invoke({
    "source_lang": "English",
    "target_lang": "Korean", 
    "text": "LangChain is a powerful framework for building AI applications."
})

print("Upstage Response:")
print(response.content)
    

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

llm = ChatUpstage(
        model="solar-pro",
        base_url="https://api.upstage.ai/v1",
        temperature=0.5
    )

# using chain
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that translates English to Korean."),
        ("human", "Translate this sentence from English to Korean. {english_text}."),
    ]
)

llm = ChatUpstage()
chain = prompt | llm

ai_message=chain.invoke({"english_text": "Hello, How are you?"})
print(ai_message.content)

Groundness Check
Groundedness Check API는 사용자가 제공한 Context(컨텍스트)에 대한 AI 어시스턴트의 응답이 실제로 그 컨텍스트에 기반하고 있는지 여부를 확인합니다.

In [6]:
from langchain_upstage import UpstageGroundednessCheck

groundedness_check = UpstageGroundednessCheck()

request_input = {
    "context": "삼성전자는 연결 기준으로 매출 74.07조원, 영업이익 10.44조원의 2024년 2분기 실적을 발표했다. 전사 매출은 전분기 대비 3% 증가한 74.07조원을 기록했다. DS부문은 메모리 업황 회복으로 전분기 대비 23% 증가하고, SDC는 OLED 판매 호조로 증가했다.",
    "answer": "삼성전자의 2024년 2분기 매출은 약 74.07조원이다.",
}

response = groundedness_check.invoke(request_input)
print(response)     

grounded


UpstageEmbeddings

In [ ]:
from langchain_upstage import UpstageEmbeddings

embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
doc_result = embeddings.embed_documents(
    ["Sung is a professor.", "This is another document"]
)
print(doc_result)

query_result = embeddings.embed_query("What does Sung do?")
print(query_result)